In [2]:
#coding=utf-8
import xlwings as xw
import re
import pandas as pd
import psycopg2
# from modules.my_libs import han_ji_chu_im_python as ji
from modules.my_libs import import_ipynb
from nbpackage import han_ji_chu_im as ji

#==========================================================
# Excel 檔案
#==========================================================

# 指定提供來源的【檔案】
file_path = 'hoo-goa-chu-im.xlsx'
wb = xw.Book(file_path)

# 指定提供來源的【工作表】；及【總列數】
source_sheet = wb.sheets['漢字注音表']
end_row = source_sheet.range('A' + str(source_sheet.cells.last_cell.row)).end('up').row
print(f'end_row = {end_row}')

#==========================================================
# 備妥程式需使用之工作表
#==========================================================
sheet_name_list = ["字庫表", "缺字表", "漢字注音表"]
#-----------------------------------------------------
# 檢查工作表是否已存在
for sheet_name in sheet_name_list:
    sheet = wb.sheets[sheet_name]
    try:
        sheet.select()
        continue
    except: 
        # CommandError 的 Exception 發生日，表工作表不存在
        # 新增程式需使用之工作表
        wb.sheets.add(name=sheet_name)

khiam_ji_sheet = wb.sheets["缺字表"]
ji_khoo_sheet  = wb.sheets["字庫表"]
chu_im_sheet   = wb.sheets["漢字注音表"]


importing Jupyter notebook from /Users/alanjui/workspace/rime/ho-lok-oe-chu-im/nbpackage/han_ji_chu_im.ipynb
end_row = 75


In [4]:
#==========================================================
# 資料庫
#==========================================================
conn = psycopg2.connect(database="alanjui", user="alanjui", host="127.0.0.1", port="5432")
cur = conn.cursor()

chu_im_index = 1
ji_khoo_index = 1
khiam_ji_index = 1
end_counter = end_row + 1

for row in range(1, end_counter):
    print(f'row = {row}')
    # 自 source_sheet 取待注音漢字
    han_ji = str(source_sheet.range('A' + str(row)).value)
    han_ji.strip()
    
    #=========================================================
    # 如是空白或換行，處理換行
    #=========================================================
    if han_ji == 'None' or han_ji == '\n':
        chu_im_sheet.range('A' + str(row)).value = '\n'
        continue
   
    #=========================================================
    # 將人個漢字，逐一加注音
    #=========================================================
    i = chu_im_index
    # 取用每一個漢字，自【字庫】資料庫查找其【注音碼】
    # 若取出之字為標點符號，則跳過，並繼續取下一個漢字。
    # piau_tiam = r"[，、：；。？！（）「」【】《》“]"
    piau_tiam = r"[；：？！\uFF0C\uFF08-\uFF09\u2013-\u2014\u2026\\u2018-\u201D\u3000\u3001-\u303F]"
    searchObj = re.search(piau_tiam, han_ji, re.M|re.I)
    if searchObj:
        continue
    
    # SQL 查詢指令：自字庫查找某漢字之注音碼
    sql =   "SELECT id, han_ji, chu_im, freq, siann, un, tiau "\
            "FROM han_ji "\
            f"WHERE han_ji='{han_ji}' "\
            "ORDER BY freq DESC;"
    cur.execute(sql)
    query_rows = cur.fetchall()
    
    # 漢字能否查到注音碼，將有不同的處理作業
    if not query_rows:
        # 問題發生：找不到漢字的注音碼
        print(f"Can not find 【{han_ji}】in Han-Ji-Khoo!!")
        khiam_ji_sheet.range('A' + str(khiam_ji_index)).value = han_ji
        khiam_ji_index += 1
        continue
    else:
        # 漢字查到注音；遇漢字有多種讀音，需於【字庫表】留紀錄
        ji_soo = len(query_rows)
        for ji_found in range(ji_soo):
            # 若查到注音的漢字，有兩個以上；則需記錄漢字的 ID 編碼
            han_ji_id = query_rows[ji_found][0]
            # 將已找到注音碼的漢字，逐一分解出：聲母、韻母、調號
            chu_im = query_rows[ji_found][2]
            chu_im_code = ji.split_chu_im(chu_im)
            siann_bu = chu_im_code[0].strip()
            un_bu = chu_im_code[1]
            tiau_ho = chu_im_code[2]

            # 取得聲母之聲母碼
            if siann_bu != "":
                siann_index = ji.get_siann_idx(siann_bu)
            
            # 取得韻母之韻母碼
            un_index = ji.get_un_idx(un_bu)
            if un_index == -1:
                # 記錄沒找到之韻母
                print(f"Can not find 【{search_han_ji}】in Han-Ji-Khoo!!")
                khiam_ji_sheet.range('A' + str(khiam_ji_index)).value = f"{han_ji}: (Un-Bu: {un_bu})"
                khiam_ji_index += 1
    
            # 取得韻母在十五音的索引編碼
            sip_ngoo_im_index = ji.get_sip_ngoo_im_idx(un_index)
    
            #=========================================================
            # 將已注音之漢字加入【漢字注音表】
            #=========================================================
            
            if ji_found == 0:
                # 處理查到的第一個漢字
                chu_im_sheet.range('B' + str(row)).value = chu_im
                chu_im_sheet.range('C' + str(row)).value = siann_bu
                chu_im_sheet.range('D' + str(row)).value = un_bu
                chu_im_sheet.range('E' + str(row)).value = tiau_ho

                chu_im_sheet.range('F' + str(row)).value = siann_index + 1
                chu_im_sheet.range('G' + str(row)).value = un_index + 1
                chu_im_sheet.range('H' + str(row)).value = sip_ngoo_im_index
            else:
                # 若查到的漢字有兩個以上
                idx = ji_khoo_index
                ji_khoo_sheet.range('A' + str(idx)).value = han_ji
            
                ji_khoo_sheet.range('B' + str(idx)).value = chu_im
                ji_khoo_sheet.range('C' + str(idx)).value = siann_bu
                ji_khoo_sheet.range('D' + str(idx)).value = un_bu
                ji_khoo_sheet.range('E' + str(idx)).value = tiau_ho

                ji_khoo_sheet.range('F' + str(idx)).value = siann_index + 1
                ji_khoo_sheet.range('G' + str(idx)).value = un_index + 1
                ji_khoo_sheet.range('H' + str(idx)).value = sip_ngoo_im_index
                
                ji_khoo_sheet.range('I' + str(idx)).value = i
                ji_khoo_sheet.range('J' + str(idx)).value = han_ji_id

                ji_khoo_index += 1

#==========================================================
# 關閉資料庫
#==========================================================
conn.close()

row = 1
row = 2
row = 3
row = 4
row = 5
row = 6
row = 7
row = 8
row = 9
Can not find 【
】in Han-Ji-Khoo!!


KeyboardInterrupt: 